In [ ]:
import numpy as np
import pandas as pd
import h5py

In [ ]:
def to_corr(x, ref):
    return (ref[0] - x) / (ref[0] - ref[1])

def to_corr_error(x, ref):
    return x / (ref[0] - ref[1])

def decimal(x,i=0):
    return (str(np.round(abs(x)*10**i, 0))[0],i) if abs(x)*10**i > 1 else decimal(x, i+1)

def get_ref(references,ref,system,ansatz):
    data = []
    if (ref,system) in references.index:
        ref_i = references.loc(0)[ref].loc(0)[system].loc(0)
        if not np.isnan(ref_i[f"{ansatz} energy"]):
            energy = 100*to_corr(ref_i[f"{ansatz} energy"],refs_exact.loc(0)[system])
            error = 100*to_corr_error(ref_i[f"{ansatz} error"],refs_exact.loc(0)[system])
            return (energy,error,1)  if 'SD' in ansatz else (energy,error,ref_i[f"{ansatz} nCSF"]) 

def conv_to_string(x, ref, csf):
    string = f"{x[0]:1.{decimal(x[1])[1]}f}({decimal(x[1])[0]})"
    if x[2]>1:
            string += f"& {int(x[2])}" + ( r'.\text{D}' if not csf else  r'.\textsc{csf}' )
    return string

In [ ]:
refs_qmc = pd.read_csv('../data/extern/small-systems-vmc.csv').set_index(['reference','system'])
refs_exact =  pd.read_csv('../data/extern/small-systems-exact.csv').set_index(['system'])
systems = ['H2', 'Be', 'B', 'LiH', 'Li2']
ansatzs =  ['SD-SJ','MD-SJ','SD-SJBF','MD-SJBF']
reference_labels = {'Brown':'[Br]', 'Casalengo':'[Ca]', 'Morales':'[Mo]', 'Rios':'[LR]', 'Seth':'[Se]', 'Toulouse':'[TU]'}
with h5py.File(f'../data/raw/data_pub_small_systems.h5', 'a') as f:
    data_paulinet={system:{ansatz:(100*to_corr(f[system][ansatz].attrs['energy'][0],refs_exact.loc(0)[system]),100*to_corr_error(f[system][ansatz].attrs['energy'][1],refs_exact.loc(0)[system]),1 if 'SD' in ansatz else 6)for ansatz in ansatzs} for system in systems}

In [ ]:
body=[]
for i,system in enumerate(systems):
    body.append("&".join([system.replace('2','$_2$')] + ['PauliNet'] + [conv_to_string(data_paulinet[system][ansatz],refs_exact.loc(0)[system],False) for ansatz in ansatzs]) + r'\\')
    for ref in reference_labels.keys():
        row = []
        add = False
        for ansatz in ansatzs:
            if get_ref(refs_qmc,ref,system,ansatz):
                row += [conv_to_string(get_ref(refs_qmc,ref,system,ansatz),refs_exact.loc(0)[system],True)]
                add = True
            else:
                row += [r'\skpdsh1'] if ansatz in ['SD-SJ','SD-SJBF'] else [r'\skpdsh2']
        if add:
            body.append(f"&".join([''] + [f"{reference_labels[ref]}"] + row)+r'\\')
    body.append(r"\midrule[0.1pt]")
body = "\n".join(body)

In [ ]:
head = [r"\begin{table*}[thb!]",
        r"\centering",
        r"\caption{\textbf{Variational correlation energy (\%) of five test systems obtained with four types of trial wave functions.}}\label{tab:energies}",
        r"\newcommand\skpdsh[1]{\multicolumn{#1}c{--}}",
        r"\newcommand\mclm[2]{\multicolumn{#1}c{#2}}",
        r"\newcommand\csf{\textsc{csf}}",
        r"\begin{tabular}{ccD..{3.6}D..{3.5}D.\,{2.5}D..{3.6}D..{3.4}D.\,{2.3}}",
        r"\toprule",
        r"& &\mclm3{Slater--Jastrow}&\mclm3{Slater--Jastrow with backflow}\\",
        r"\cmidrule(lr){3-5}\cmidrule(lr){6-8}&ref.$^a$ & \mclm1{SD$^b$} & \mclm2{MD$^c$} & \mclm1{SD} & \mclm2{MD}\\",
        r"\midrule"]
head = "\n".join(head)

In [ ]:
tail = [r"\bottomrule",
        r"\end{tabular}",        
        r"\footnotesize",
        r"\begin{minipage}{0.65\linewidth},",
        r"$^a$The parenthesized letters encode the following references: [Ca]--\citet{CasalegnoJCP03}, [TU]--\citet{ToulouseJCP08}, [Mo]--\citet{MoralesJCTC12}, [LR]--\citet{LopezRiosPRE06}, [Br]--\citet{BrownJCP07}, [Se]--\citet{SethJCP11}.",
        r"$^b$Single-determinant ansatz.",
        r"$^c$Multi-determinant ansatz. ``D'' denotes a determinant, ``\textsc{csf}'' denotes a configuration state function.",
        r"\end{minipage}",
        r"\end{table*}"]
tail = "\n".join(tail)

In [ ]:
print(head)
print(body)
print(tail)